In [14]:
pip install --upgrade google-cloud-language

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 2.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Part 1/2: Audio file Transcription

In [12]:
import os
from google.cloud import storage, speech
from google.protobuf.json_format import MessageToJson
import time

# Set up authentication
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/pawanbtw/Downloads/sound-works-425306-f1-506b00b9120f.json"

def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the Google Cloud Storage bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

def transcribe_audio(gcs_uri):
    """Transcribes the audio file specified by the GCS URI."""
    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.MP3,  # Adjust encoding if necessary
        sample_rate_hertz=44100,  # Adjust sample rate if necessary
        language_code='en-US'
    )

    operation = client.long_running_recognize(config=config, audio=audio)
    print('Waiting for operation to complete...')
    
    # Polling mechanism
    while not operation.done():
        print('Operation not completed yet. Waiting for 60 seconds...')
        time.sleep(60)
    
    response = operation.result(timeout=3600)  # 1-hour timeout

    transcript = ''.join([result.alternatives[0].transcript for result in response.results])
    return transcript

# Replace with your bucket name, local file path, and desired GCS file name
bucket_name = "senti-bucket22"
source_file_name = "/Users/pawanbtw/Downloads/meeting_demo.mp3"
# Replace with your desired GCS file name
destination_blob_name = "audio/meeting_demo.mp3"

# Upload the file to GCS
upload_to_gcs(bucket_name, source_file_name, destination_blob_name)

# Construct the GCS URI
gcs_uri = f"gs://{bucket_name}/{destination_blob_name}"

# Transcribe the audio file from GCS
transcribed_text = transcribe_audio(gcs_uri)
print("Transcription:")
print(transcribed_text)





File /Users/pawanbtw/Downloads/meeting_demo.mp3 uploaded to audio/meeting_demo.mp3.
Waiting for operation to complete...
Operation not completed yet. Waiting for 60 seconds...
Operation not completed yet. Waiting for 60 seconds...
Operation not completed yet. Waiting for 60 seconds...
Operation not completed yet. Waiting for 60 seconds...
Operation not completed yet. Waiting for 60 seconds...
Operation not completed yet. Waiting for 60 seconds...
Operation not completed yet. Waiting for 60 seconds...
Transcription:
Coreg awesome yeah so I see how long you were wanting to jump into the topic for a remodel. So we'll go ahead and do that yep to Jessica using the release from this table inside of the old 70 show again for migrating existing policies into the new cable so once that's done we can complete before we enable the feature flag would it make sense to also implemented for the pipeline execution policies because then we wouldn't would need to have a baker of migration for that ineff

# # Part 2/2: Teams_meeting_sentiment

In [16]:
from google.cloud import language_v1

def analyze_sentiment(text_content):
    """Analyzes the sentiment of the text content."""
    client = language_v1.LanguageServiceClient()

    document = language_v1.Document(content=text_content, type_=language_v1.Document.Type.PLAIN_TEXT)
    response = client.analyze_sentiment(request={'document': document})

    # Get overall sentiment score
    sentiment = response.document_sentiment
    sentiment_score = sentiment.score
    sentiment_magnitude = sentiment.magnitude

    return sentiment_score, sentiment_magnitude

# Perform sentiment analysis on the transcription
sentiment_score, sentiment_magnitude = analyze_sentiment(transcribed_text)

# Determine sentiment label based on sentiment score
if sentiment_score > 0.2:
    sentiment_label = 'positive'
elif sentiment_score < -0.2:
    sentiment_label = 'negative'
else:
    sentiment_label = 'neutral'

print(f"Sentiment: {sentiment_label}")
print(f"Sentiment Score: {sentiment_score}")
print(f"Sentiment Magnitude: {sentiment_magnitude}")


Sentiment: neutral
Sentiment Score: -0.10000000149011612
Sentiment Magnitude: 6.400000095367432
